In [4]:
from langchain_community.tools import tool
from dotenv import load_dotenv
import requests
import os

In [3]:
load_dotenv()

EXCHANGE_API = os.environ["EXCHANGE_API"]

1. Tool Creation

In [24]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

# get the conversion rate between target currency and base currency
@tool
def get_conversion_rate(base_currency:str,target_currency:str) -> float:
    """
    This function fetches the currency conversion rate, given a base currency and target currency
    """
    
    # define the url
    url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_API}/pair/{base_currency}/{target_currency}"
    
    # get the response from the url
    response = requests.get(url=url).json()
    
    # return the conversion rate
    return response['conversion_rate']    


# another tool to convert the base amount, into target amount by multiplying with given factor.
@tool
def convert_currency(base_currency_value:int,conversion_rate:Annotated[float,InjectedToolArg])->float: # i am telling the llm to not assume values for the conversion rate, and I will provide the conversion rate, after calling earlier tools
    """
    This function multiplies the base currency value by the conversion rate
    to obtain target currency value. 
    """
    
    return base_currency_value * conversion_rate

In [25]:
get_conversion_rate.invoke({"base_currency":"INR","target_currency":"USD"})

0.01099

2. Bind the tools with llm

In [86]:
from langchain_groq import ChatGroq

# create an llm
llm = ChatGroq(model="llama-3.3-70b-versatile")

# bind the tools with llm now
llm_currency_convertor = llm.bind_tools([get_conversion_rate,convert_currency])

3. Tool calling

In [87]:
from langchain_core.messages import HumanMessage

# maintain a chat_history
chat_history = [HumanMessage("Hello, do you know the conversion rate for dollar to inr, and based on that can you convert 10 dollars to inr?")]

# invoke the llm with this message
response1 = llm_currency_convertor.invoke(chat_history)
# and record the response into chat history
chat_history.append(response1)

In [88]:
chat_history

[HumanMessage(content='Hello, do you know the conversion rate for dollar to inr, and based on that can you convert 10 dollars to inr?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'abd85a7r5', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_rate'}, 'type': 'function'}, {'id': 'hgytvsmyv', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert_currency'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 354, 'total_tokens': 393, 'completion_time': 0.096969547, 'completion_tokens_details': None, 'prompt_time': 0.018580092, 'prompt_tokens_details': None, 'queue_time': 0.059330528, 'total_time': 0.115549639}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019

In [89]:
response1.tool_calls

[{'name': 'get_conversion_rate',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'abd85a7r5',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'base_currency_value': 10},
  'id': 'hgytvsmyv',
  'type': 'tool_call'}]

In [90]:
get_conversion_rate.invoke(response1.tool_calls[0])

ToolMessage(content='90.9589', name='get_conversion_rate', tool_call_id='abd85a7r5')

In [91]:
for tool_call in response1.tool_calls:
    # if its the 1st tool, then get the conversion rate first
    if tool_call['name']=="get_conversion_rate":
        # fetch the conversion rate
        tool_msg1 = get_conversion_rate.invoke(tool_call)
        conversion_rate = tool_msg1.content
        # and append the tool_msg to the chat_history
        chat_history.append(tool_msg1)
    
    # and for the second tool call
    if tool_call['name']=="convert_currency":
        # then add conversion rate to the current 'args' 
        tool_call['args']['conversion_rate'] = float(conversion_rate)
        # set the new tool_call in ai_message
        response1.tool_calls[1] = tool_call
        # now execute the second tool
        tool_msg2 = convert_currency.invoke(tool_call)
        # and append it to chat_history
        chat_history.append(tool_msg2)

In [92]:
chat_history

[HumanMessage(content='Hello, do you know the conversion rate for dollar to inr, and based on that can you convert 10 dollars to inr?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'abd85a7r5', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_rate'}, 'type': 'function'}, {'id': 'hgytvsmyv', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert_currency'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 354, 'total_tokens': 393, 'completion_time': 0.096969547, 'completion_tokens_details': None, 'prompt_time': 0.018580092, 'prompt_tokens_details': None, 'queue_time': 0.059330528, 'total_time': 0.115549639}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019

In [93]:
final_response = llm_currency_convertor.invoke(chat_history)
print(final_response)

content='' additional_kwargs={'tool_calls': [{'id': 'b8mknhsqb', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_rate'}, 'type': 'function'}, {'id': 'wvqf1498x', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert_currency'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 430, 'total_tokens': 469, 'completion_time': 0.062482144, 'completion_tokens_details': None, 'prompt_time': 0.020881077, 'prompt_tokens_details': None, 'queue_time': 0.059226753, 'total_time': 0.083363221}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bdb45-d01f-7e73-8da5-cdf71575096a-0' tool_calls=[{'name': 'get_conversion_rate', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'b8mknhsqb', 'type': 'tool_call'}, {'name': 'con